In [ ]:
# Note: It is important to not call any other function such as store.list_dir
# after creating the store and before creating the group, as this will lead to an error
# "got Future <Future pending> attached to a different loop"

# async for item in store.list_dir(""):
#     print(item)

In [ ]:
# mamba install h5py zarr --channel conda-forge

# There is a bug with the newest versions of botocore (1.36 and 1.37)
# mamba install botocore=1.35

In [ ]:
import os

import h5py
import zarr
from s3fs import S3FileSystem

In [ ]:
session = "7T1027"

In [ ]:
h5_preproc_fname = (
    "/home/projects/qrage/sessions/%s/preproc/mz_me_mpnrage3d_grappa.h5" % session
)
zarr_preproc_fname = (
    "qrage/sessions/%s/preproc.zarr/mz_me_mpnrage3d_grappa.zarr" % session
)

In [ ]:
fs = S3FileSystem(
    anon=False,
    endpoint_url="https://s3.fz-juelich.de",
    key=os.getenv("AWS_ACCESS_KEY_ID"),
    secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
    asynchronous=True,
)

In [ ]:
store = zarr.storage.FsspecStore(
    fs,
    path=zarr_preproc_fname,
)

In [ ]:
root = zarr.create_group(
    store=store,
    overwrite=True,
)

In [ ]:
with h5py.File(h5_preproc_fname, "r") as f:
    for key in f.keys():
        print(key)
        array = root.create_array(
            name=key,
            shape=f[key].shape,
            chunks=f[key].chunks,
            dtype=f[key].dtype,
            overwrite=True,
        )
        array[:] = f[key][:]

In [ ]:
root.tree()